<a href="https://colab.research.google.com/github/AbdulrhmnGhanem/bdd100k-trajectories/blob/main/nth_frame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we are going to desgin the algorithm to get the frame at the `nth` displacement. We haven't settled on the `SAMPLING_DISTANCE` value, but I will prototype with `SAMPLING_DISTANCE = 10`.

In [1]:
# Reproducible-ish dependencies
!pip install \
    pandas==1.3.5 \
    requests==2.23.0 \
    geopy==1.17.0 \
    imageio==2.4.1 \
    Pillow==7.1.2

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from getpass import getpass
API_KEY = getpass('Enter the google maps API key: ')

Enter the google maps API key: ··········


## Read the stored `selected_trajectories` file

> The selection criteria was 
> 
> $  50m \le total \ displacement \ along \ the \ trajectory \le 2500m$
>

> ### Division
* Assuming we managed to divide the trajectories into $50m$ sub-trajectories.
* We have the speed at each point, and we can assume constant speed between adjacent points of the average speed at these point $v_{n → n+1} = {{v_{n} + v_{n+1} \over 2}, \ n \in \{1, \ 2, \ 3, \ \ldots, \lfloor{trajectory \ distace \over SAMPLING\_DISTANCE} \}}$.
* The time between adjacent points is $1s$.
* To get the time of corresponding frame for the $nth$ point we can use this formula $t_n={d \times n \over v_{n → n+1}} , \ n \in \{1, \  2, \  3, \  \ldots, \lfloor{trajectory \ distace \over SAMPLING\_DISTANCE} \}$.
* Then pass the list of $t_n$s to `ffmpeg` to extract the frames. 

> https://colab.research.google.com/drive/1RVCX9pociA3bU9IEeJNz0xaboCKWqYoq#scrollTo=zsx25ZJ-8PTK&line=9&uniqifier=1

In [3]:
import pickle

with open('/content/drive/MyDrive/selected_trajectories.pkl', 'rb') as f:
  selected_trajectories = pickle.load(f) 

In [4]:
from typing import List, Tuple
from itertools import accumulate
from functools import lru_cache
import io

import geopy.distance
import PIL
import imageio
import requests


ROUNDING_NDIGITS = 2

def trajectory_to_points(trajectory: Tuple[str, List]) -> List[Tuple[float, float]]:
  "Return list of GPS points from a trajectory"
  return [(point["latitude"], point["longitude"]) for point in trajectory[1]]

def get_trajectory_running_distance(points: List[Tuple[float, float]]) -> List[float]:
  """
  Get the running distance of the trajectory at each second of the video
  :param: points - list of GPS points, [](latitude, longitude)
  """
  # prepadding a zero at the beginning because the running distance at the start is zero.
  adjacent_distance = [0] + [geopy.distance.vincenty(p1, p2).meters for p1, p2 in zip(points, points[1:])]
  running_distance = accumulate(adjacent_distance)
  return [round(d, ROUNDING_NDIGITS) for d in running_distance]


def mark_trajectory(points, *,zoom, scale, size):
  base_url = "https://maps.googleapis.com/maps/api/staticmap"
  key=API_KEY
  markers = '|'.join(f"{x}, {y}" for x, y in points)

  img_url = f"{base_url}?key={key}&zoom={zoom}&scale={scale}" \
                f"&size={size}&maptype=satellite&format=png" \
                f"&visual_refresh=true&markers={markers}"
  return PIL.Image.open(
      io.BytesIO(requests.get(img_url).content)).convert('RGB')

@lru_cache(maxsize=1)
def load_total_distance_df():
  with open('/content/drive/MyDrive/total_distance_df.pkl', 'rb') as f:
    df = pickle.load(f)
  return df

@lru_cache()
def get_trajectory_total_distance(trajectory_name: str):
  df = load_total_distance_df()
  return round(df.loc[trajectory_name][0], ROUNDING_NDIGITS)

def get_average_speeds(trajectory):
  """
  vn→n+1
  The speed beween two adjacent points
  """
  points_speed = [p["speed"] for p in sample_trajectory[1]]
  return [round((s1+s2)/2, ROUNDING_NDIGITS) for s1, s2 in zip(points_speed, points_speed[1:])]

In [5]:
from numpy.testing import assert_almost_equal

sample_trajectory = selected_trajectories[0]

trajectory_running_distance = get_trajectory_running_distance(trajectory_to_points(sample_trajectory))
total_distance_using_running_distance = trajectory_running_distance[-1]
total_distance_pre_computed = get_trajectory_total_distance(sample_trajectory[0])

assert_almost_equal(total_distance_using_running_distance, total_distance_pre_computed)

So we can concloude the running average calculations are right! Now let's calcualte figure out how to get the time of the frame.

In [6]:
get_average_speeds(sample_trajectory)

[6.97,
 8.97,
 10.7,
 11.93,
 12.38,
 12.28,
 12.08,
 11.62,
 11.0,
 10.5,
 10.02,
 8.99,
 7.82,
 7.33,
 7.12,
 6.93,
 6.7,
 6.15,
 5.02,
 3.28,
 1.14,
 0.81,
 2.1,
 2.62,
 2.64,
 2.73,
 2.3,
 2.22,
 3.72,
 5.22,
 6.81,
 8.47,
 9.57,
 10.63,
 11.18,
 11.47,
 12.0,
 12.61,
 12.86]